# **Test**

#### **#13 - Patch**

In [ ]:
# split predicting image into patches
#import os

from osgeo import gdal

# set path
in_path = pred_dir + 'Raw/MCC/AnhTuyen/'
input_filename = 'BDA_MuCangChai_GeoLLWGS84_P1.tif'
#---
out_path = pred_dir + 'Patches/MCC/AnhTuyen/BDA_MuCangChai_GeoLLWGS84_P1/'

if not os.path.exists(out_path):
    os.mkdir(out_path)

output_filename = 'MCC_P1_'

# tile creation
tile_size_x = 128
tile_size_y = 128

# read band for getting dimensions
ds = gdal.Open(in_path + input_filename)
band = ds.GetRasterBand(1)
xsize = band.XSize
ysize = band.YSize
# iterate the image
for i in range(0, xsize + 2*tile_size_x - xsize%tile_size_x, tile_size_x):
    for j in range(0, ysize + 2*tile_size_y - ysize%tile_size_y, tile_size_y):
        com_string = "gdal_translate -of GTIFF -srcwin " + str(i)+ ", " + str(j) + ", " + str(tile_size_x) + ", " + str(tile_size_y) + " " + str(in_path) + str(input_filename) + " " + str(out_path) + str(output_filename) + str(i) + "_" + str(j) + ".tif"
        os.system(com_string)
print('Separation done!')      

In [ ]:
# split predicting image into patches
from utils import for_test
from osgeo import gdal
import glob

# set path
in_path = pred_dir + 'Raw/MCC/AnhTuyen/'
input_filename = 'BDA_MuCangChai_GeoLLWGS84_P1.tif'
#---
out_path = pred_dir + 'Patches/MCC/AnhTuyen/BDA_MuCangChai_GeoLLWGS84_P1/'

if not os.path.exists(out_path):
    os.mkdir(out_path)

output_filename = 'MCC_P1_'

# patching
for_test.patching(in_path, input_filename, patch_size=128)

#### **#14 - Normalize**

In [ ]:
# call out predicting image
img_pred_paths = glob.glob(pred_dir+'Patches/MCC/AnhTuyen/BDA_MuCangChai_GeoLLWGS84_P1/*.tif')

stacked_img_norms, meta_out = for_test.normalize_patches(img_pred_paths)

#### **#15 - Predict**

In [ ]:
# predict
import tensorflow as tf # call out the trained model

test_model = tf.keras.models.load_model('Output/Model/unet.h5') # load model

# stack normalized patches
stack_norms = np.stack(img_norms)
# save stack
np.save(pred_dir+'Test/Output/MCC/AnhTuyen/BDA_MuCangChai_GeoLLWGS84_P1/stack_for_pred.npy', stack_norms)

# predict by test image
any_pred = test_model.predict(stack_norms[:,:,:,0:3], batch_size=30)
# save pure predictions
np.save(pred_dir+ 'Test/Output/MCC/AnhTuyen/BDA_MuCangChai_GeoLLWGS84_P1/pure_pred.npy', any_pred)

In [ ]:
# call out model
import tensorflow as tf
trained_model = tf.keras.models.load_model('Output/Model/unet.h5') # load model

# save stack
np.save(pred_dir+'Output/MCC/AnhTuyen/BDA_MuCangChai_GeoLLWGS84_P1/stack_for_pred.npy', stack_norms)

# predict by test image
predictset, predict_results = model._predict_(trained_model, stack_norms[:,:,:,0:3], batch_size=30)
# save pure predictions
np.save(pred_dir+ 'Output/MCC/AnhTuyen/BDA_MuCangChai_GeoLLWGS84_P1/pure_pred.npy', predict_results)

#### **#16 - Save predicting results**

In [ ]:
# save the predicted results
for i in range(len(predict_results)):
    name = img_pred_paths[i].split('Patches')[0]+'Output/'+img_pred_paths[i].split('Patches')[-1].split('\\')[0]+'/'+img_pred_paths[i].split('\\')[-1]
    with rio.open(name, "w", **meta_out[i]) as dest:
        # reclassify the predicted result
        result = np.argmax(predict_results[i])
        dest.write(result[:,:,1], indexes=1)
    dest.close()


In [ ]:
for_test.save_prediction(predict_results, img_pred_paths)

#### **#17 - Merge predicting patches**

In [ ]:
# merge predicted results
import glob
from rasterio.merge import merge

pre_re_paths = glob.glob(pred_dir+'Test/Output/MCC/AnhTuyen/BDA_MuCangChai_GeoLLWGS84_P3/*.tif') # find the predicted results' locations

src_files_to_mosaic = [] # list for opening rasters

# open rasters
for src in pre_re_paths:
    img = rio.open(src)
    src_files_to_mosaic.append(img)
    
# copy the metadata
out_meta = img.meta.copy()

mosaic, out_trans = merge(src_files_to_mosaic) # merge predicted results

# write the mosaic raster to disk
# update the metadata
out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                 "crs": img.crs,
                 "dtype": rio.uint8,
                 "nodata": 0
                 }
                )

with rio.open(pred_dir+'Test/Output/MCC/AnhTuyen/Mosaic/pred_BDA_MuCangChai_GeoLLWGS84_P3.tif', "w", **out_meta) as dest:
    dest.write(mosaic)
    
dest.close()

In [ ]:
# mosaicking
pre_re_paths = glob.glob(pred_dir+'Output/MCC/AnhTuyen/BDA_MuCangChai_GeoLLWGS84_P1/*.tif') # find the predicted results' locations

# mosaic
for_test.mosaicking_pred(pre_re_paths, meta_out)